In [31]:
from transformers import BertModel, BertTokenizer
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
import re
import json
import numpy as np
import pandas as pd


In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese') 

In [8]:
doc1 = [
    '这是一个测试文段,测试特殊字符',
    '我们再写一个测试字符串',
    '以便进行一些长度不同的新的测试',
    '',
    ''
    ]

doc2 = [
    '我们写了第二个测试文档',
    '以便处理文本长度不一致的问题'
]
string = [doc1, doc2]
token_str = tokenizer(
    #string,
    doc1,
    padding = True, 
    return_tensors = 'pt',
    max_length=30,
    truncation=True
    )
for k in token_str.keys():
    print(k, token_str[k])
tokenizer.decode(token_str['input_ids'][0])

input_ids tensor([[ 101, 6821, 3221,  671,  702, 3844, 6407, 3152, 3667,  117, 3844, 6407,
         4294, 3654, 2099, 5016,  102],
        [ 101, 2769,  812, 1086, 1091,  671,  702, 3844, 6407, 2099, 5016,  706,
          102,    0,    0,    0,    0],
        [ 101,  809,  912, 6822, 6121,  671,  763, 7270, 2428,  679, 1398, 4638,
         3173, 4638, 3844, 6407,  102],
        [ 101,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0],
        [ 101,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0]])
token_type_ids tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

'[CLS] 这 是 一 个 测 试 文 段, 测 试 特 殊 字 符 [SEP]'

In [11]:
d = tokenizer.get_vocab()

In [4]:
t = torch.zeros(3, 30).type(torch.long)
t[0:3, 0:17] = token_str['input_ids']
print(t)

tensor([[ 101, 6821, 3221,  671,  702, 3844, 6407, 3152, 3667,  117, 3844, 6407,
         4294, 3654, 2099, 5016,  102,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [ 101, 2769,  812, 1086, 1091,  671,  702, 3844, 6407, 2099, 5016,  706,
          102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [ 101,  809,  912, 6822, 6121,  671,  763, 7270, 2428,  679, 1398, 4638,
         3173, 4638, 3844, 6407,  102,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]])


In [5]:
bert = BertModel.from_pretrained('bert-base-chinese')
bert_output = bert(
    input_ids = token_str['input_ids'],
    attention_mask = token_str['attention_mask'],
    #token_type_ids = token_str['token_type_ids']
)

In [41]:
t1 = torch.arange(9).view(-1,3)
t2 = torch.arange(8).view(-1,4)
l = [3,2]
tp = pad_sequence([t1,t2], batch_first=True)
ptp = pack_padded_sequence(tp, l, batch_first=True)
print(t1,t2,tp, sep='\n')
print(ptp)
bert(ptp.data).pooler_output.shape

RuntimeError: The expanded size of the tensor (3) must match the existing size (4) at non-singleton dimension 1.  Target sizes: [2, 3].  Tensor sizes: [2, 4]

In [19]:
test_input = torch.tensor([[1,2,3,4,5,6,7,8,0,0,0,0]]) #.view(3,4)
mask = (torch.tensor([[1,2,3,4,5,6,7,8,0,0,0,0]]) > 0).type(torch.int)
#test_mask = torch.zeros(3,4)
#print(test_input, test_mask)
t1 = bert(input_ids = test_input).pooler_output

test_input = torch.tensor([[1,2,3,4,5,6,7,8,0,0]])#.view(3,4)
mask = (torch.tensor([[1,2,3,4,5,6,7,8,0,0]]) > 0).type(torch.int)
t2 = bert(input_ids = test_input).pooler_output
print(torch.allclose(t1,t2))


False


In [18]:
mask = (torch.tensor([[1,2,3,4,5,6,7,8,0,0,0,0]]) > 0).type(torch.int)
print(mask)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]], dtype=torch.int32)


In [32]:
test_input = torch.zeros(3,4).type(torch.long)
test_input[:,0] = 101
test_input[:,-1] = 102
test_mask = torch.zeros(3,4)
print(test_input, test_mask)
print(bert(input_ids = test_input).pooler_output)

tensor([[101,   0,   0, 102],
        [101,   0,   0, 102],
        [101,   0,   0, 102]]) tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[ 0.9827,  0.9999,  0.9666,  ..., -0.9996, -0.9377,  0.7818],
        [ 0.9827,  0.9999,  0.9666,  ..., -0.9996, -0.9377,  0.7818],
        [ 0.9827,  0.9999,  0.9666,  ..., -0.9996, -0.9377,  0.7818]],
       grad_fn=<TanhBackward>)


In [7]:
bert_output.pooler_output

tensor([[ 0.9998,  0.9998,  0.9983,  ..., -0.9996, -0.9990, -0.5405],
        [ 0.9987,  0.9998,  0.9990,  ..., -0.9975, -0.9903, -0.7868],
        [ 0.9994,  1.0000,  0.9030,  ..., -0.9949, -0.9962,  0.8239]],
       grad_fn=<TanhBackward>)

In [30]:
t1 = torch.arange(0,16).view(-1,4)
t1 = torch.cat([t1, torch.zeros(1,4)], dim = 0)

t2 = torch.arange(0,12).view(-1,4)
t2 = torch.cat([t2, torch.zeros(2,4)], dim = 0)

t = torch.stack([t1,t2], dim=0)
l = [4,3]
ps = pack_padded_sequence(t, l, batch_first=True)

print(ps)

PackedSequence(data=tensor([[ 0.,  1.,  2.,  3.],
        [ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [ 8.,  9., 10., 11.],
        [12., 13., 14., 15.]]), batch_sizes=tensor([2, 2, 2, 1]), sorted_indices=None, unsorted_indices=None)


In [ ]:
torch.pad

In [19]:
for k in bert_output.__dict__.keys():
    print(k,'\t', bert_output.__dict__[k])

last_hidden_state 	 tensor([[[-0.6608,  0.4741,  0.6114,  ...,  0.2885,  0.0974,  0.1135],
         [-0.3507,  0.6145,  1.2518,  ..., -0.2922, -0.5560,  0.0368],
         [-0.2561, -0.0083,  1.3459,  ..., -0.1950,  0.3284,  0.1385],
         [-0.7835,  0.7552,  0.0676,  ...,  0.2537,  0.1146, -0.1255],
         [-0.1400,  0.4187,  1.0834,  ..., -0.0825, -0.4506,  0.5955]],

        [[-0.0362,  0.7508, -0.3406,  ...,  0.2986, -0.3089, -0.1495],
         [-0.0112, -0.1314, -0.3386,  ..., -0.7059, -0.5913,  0.0548],
         [ 0.3185, -0.6601, -0.1627,  ..., -0.0639,  0.2416,  0.0647],
         [ 0.2119,  0.9714, -0.1111,  ..., -0.3018, -0.5880, -0.1860],
         [-0.6646,  0.1629, -0.0872,  ..., -0.3346, -0.0014, -0.0975]],

        [[ 0.1191,  0.5861,  0.6062,  ...,  0.5815,  0.4746, -0.2843],
         [ 0.2881,  0.2342,  1.1484,  ..., -0.7969, -0.3408,  0.1214],
         [ 0.2756,  0.3449,  0.7496,  ...,  0.5526,  0.5725,  0.1504],
         [ 0.2385,  0.6318,  0.2960,  ..., -0.2246,  

In [34]:
lhs = bert_output['pooler_output']
lhs.shape

torch.Size([3, 768])

In [63]:
INPUT_SIZE = 768
HIDDEN_SIZE = 16
NUM_LAYERS = 1

bi_lstm = nn.LSTM(
    input_size = INPUT_SIZE,
    hidden_size = HIDDEN_SIZE,
    batch_first = True,
    bidirectional = True
)
output,(_,_) = bi_lstm(
    bert_output['pooler_output'].reshape(1,3,768)
)
print(output[:,-1,:].shape)

torch.Size([1, 32])


## functions

In [ ]:
def get_sample_para():
    p = './data/doc_quality_data_train.json'
    para_list = []
    with open(p,'r') as f:
        for i in range(3):
            l = json.loads(f.readline())
            para_list.append(l['body'])
    return para_list
para_list = get_sample_para()

In [111]:
def _cut_doc(para):
    '''
    remain to modify: 
    1. maybe we can delete all the punctuations
    '''
    para = re.sub('\s*([。！？;\?])([^”’])\s*', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('\s*(\.{6})([^”’])\s*', r"\1\n\2", para)  # 英文省略号
    para = re.sub('\s*(\…{2})([^”’])\s*', r"\1\n\2", para)  # 中文省略号
    para = re.sub('\s*([。！？\?][”’])([^，。！？\?])\s*', r'\1\n\2', para)
    para = re.sub(r'\s+', r'\n', para)
    para = para.rstrip()  # 段尾如果有多余的\n就去掉它
    return para.split("\n")

_cut_doc(para_list[0])

['篮球——CBA第四阶段：辽宁本钢迎战吉林九台农场银行',
 '新华社照片，诸暨（浙江），2021年3月26日',
 '（体育）（13）篮球——CBA第四阶段：辽宁本钢迎战吉林九台农场银行',
 '3月26日，辽宁本钢队主教练杨鸣在比赛中指挥。',
 '当日，在浙江诸暨举行的2020-2021赛季中国男子篮球职业联赛（CBA）第四阶段第48轮比赛中，辽宁本钢队对阵吉林九台农商银行队。',
 '新华社记者',
 '孟永民',
 '摄']

In [113]:
def _cut_docs(docs):
    '''
    docs: list of docs
    '''
    return [_cut_doc(d) for d in docs]
docs_seq = _cut_docs(para_list)

In [130]:
def get_lengths(docs_sequences):
    doc_lengths = [len(d) for d in docs_sequences]
    sent_lengths = []
    for d in docs_sequences:
        sent_lengths.append([len(y) for y in d])
    max_sent_length = max([max(y) for y in sent_lengths])
    max_doc_length = max(doc_lengths)

    return max_sent_length, max_doc_length, sent_lengths, doc_lengths
    
print(get_lengths(docs_seq))

(68, 44, [[26, 23, 34, 23, 66, 5, 3, 1], [43, 38, 29, 28, 25, 13, 15, 48, 26, 33, 38, 32, 30, 36, 37, 47, 39, 18, 19, 9, 25, 35, 29, 68, 27, 37, 20, 21, 34, 20, 56, 23, 40, 17, 51, 52, 34, 16, 62, 28, 23, 55, 31, 1], [10, 3, 55]], [8, 44, 3])


In [153]:
def bert_preprocess(docs, bertTokenizer, bert_max_length = 512):
    #docs_seq = _cut_docs(docs)
    input_ids = []
    attention_mask = []
    token_type_ids = []
    for d in docs_seq:
        tk = bertTokenizer(d, return_tensors = 'pt', padding = True)
        input_ids.append(tk['input_ids'])
        attention_mask.append(tk['attention_mask'])
        token_type_ids.append(tk['token_type_ids'])
    return input_ids, attention_mask, token_type_ids

input_ids, attention_mask, token_type_ids = bert_preprocess(string, tokenizer)

In [6]:
import numpy as np
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [8]:
V = 5 # 词典大小V=5
words = range(5)

In [9]:
data = [[1,2,3], [0,0,1,2,4], [1,0,0,4], [4,4,2]]
data[0]

[1, 2, 3]

In [10]:
# 首先按照序列长度由大到小排序
data = sorted(data, key=len, reverse=True)

lengths = [len(ins) for ins in data] # 每条训练样本的序列长度
T = len(data[0]) # 最大的序列长度
B = len(data)  # batch_size
T

5

In [12]:
def convert_one_hot_encoding(input, T, V):
    data = np.zeros((T, V)) # (T, V)
    data[np.array(range(len(input))), np.array(input)] = 1
    
    return data

In [13]:
data = [convert_one_hot_encoding(seq, T, V) for seq in data]

In [14]:
data


[array([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1.]]),
 array([[0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.]]),
 array([[0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])]